In [57]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from torch.utils.data import Dataset, DataLoader
import math
import scipy.stats as stats
from sklearn.preprocessing import MinMaxScaler
import time

In [58]:
"亂數種子"
np.random.seed(20)

def mkdir(path):
    #判斷目錄是否存在，存在：True、不存在：False
    folder = os.path.exists(path)
    if not folder:
        os.makedirs(path)

filename = []
#bad_dataset = [1,2,10,30,49,52,66,83,97,101,114,117,121] #<=137是清福的長輩、<=67是65歲以上長輩
bad_dataset = [1,2,10,30,49,52,66,83,94,97,101,107,114,117,121,124] #<=137是清福的長輩、<=67是65歲以上長輩、扣掉離群資料
y_data = pd.read_csv('ScaleScore/BBS.csv')

"取csv檔並排除有問題的受測者"
DatasetPath = "BalanceDataSet_150/"

filepath = os.listdir(DatasetPath)
for files in filepath:
    if files.endswith(".csv"):  #只留副檔名為.csv檔
        #if int(files[0:3]) < 137 and int(files[0:3]) > 65 and not (int(files[0:3]) in bad_dataset):    #只使用長輩資料(61位)
        if int(files[0:3]) < 137 and not (int(files[0:3]) in bad_dataset):  #只使用年輕人與舊庄長輩資料(120位)
            filename.append(files)

In [ ]:
DEVICE=torch.device("cuda" if torch.cuda.is_available() else "cpu")
"cuda" if torch.cuda.is_available() else "cpu"

In [60]:
def split_and_shuffle(subject_gait_data_x, subject_gait_data_y,val_times,subject_size,i):
    x_test_subject = subject_gait_data_x[math.floor(i/val_times*subject_size) : math.floor((i+1)/val_times*subject_size)]
    y_test_subject = subject_gait_data_y[math.floor(i/val_times*subject_size) : math.floor((i+1)/val_times*subject_size)]

    if i == 0:
        x_train_subject = subject_gait_data_x[math.floor((i+1)/val_times*subject_size) : subject_size]
        y_train_subject = subject_gait_data_y[math.floor((i+1)/val_times*subject_size) : subject_size]
    elif i == val_times-1:
        x_train_subject = subject_gait_data_x[0 : math.floor(i/val_times*subject_size)]
        y_train_subject = subject_gait_data_y[0 : math.floor(i/val_times*subject_size)]
    else:
        x_train_subject_1 = subject_gait_data_x[0 : math.floor(i/val_times*subject_size)]
        x_train_subject_2 = subject_gait_data_x[math.floor((i+1)/val_times*subject_size) : subject_size]
        x_train_subject = x_train_subject_1 + x_train_subject_2

        y_train_subject_1 = subject_gait_data_y[0 : math.floor(i/val_times*subject_size)]
        y_train_subject_2 = subject_gait_data_y[math.floor((i+1)/val_times*subject_size) : subject_size]
        y_train_subject = np.concatenate([y_train_subject_1 , y_train_subject_2])

    x_train = []
    y_train = []
    for j in range(len(x_train_subject)):
        for k in range(len(x_train_subject[j])):
            x_train.append(x_train_subject[j][k])
            y_train.append(y_train_subject[j])
    x_train = np.array(x_train)
    y_train = np.array(y_train)

    x_test = []
    y_test = []
    for j in range(len(x_test_subject)):
        for k in range(len(x_test_subject[j])):
            x_test.append(x_test_subject[j][k])
            y_test.append(y_test_subject[j])
    x_test = np.array(x_test)
    y_test = np.array(y_test)

    "打亂訓練資料"
    index = np.random.permutation(x_train.shape[0])
    x_train = x_train[index]
    y_train = y_train[index]

    "打亂測試資料"
    index = np.random.permutation(x_test.shape[0])
    x_test = x_test[index]
    y_test = y_test[index]

    return x_train, y_train, x_test, y_test

In [61]:
class FeatureExtractor(nn.Module): 
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        self.con1=nn.Conv1d(in_channels=9, out_channels=32, kernel_size=11,padding=5)
        self.fc1=nn.BatchNorm1d(32)
        self.fc2=nn.ReLU()
        self.con2=nn.Conv1d(in_channels=32, out_channels=32, kernel_size=11,padding=5)
        self.fc3=nn.BatchNorm1d(32)
        self.fc4=nn.ReLU()
        self.fc5=nn.MaxPool1d(3)

    def forward(self,x):
        x=self.con1(x)
        x=self.fc1(x)
        x=self.fc2(x)
        x=self.con2(x)
        x=self.fc3(x)
        x=self.fc4(x)
        x=self.fc5(x)


        return x

In [62]:
class Classifier(nn.Module): 
    def __init__(self):
       super(Classifier, self).__init__()
       self.lstm1 = nn.LSTM(32, 64, batch_first=True)
       self.dropout1 = nn.Dropout(0.5)
       self.lstm2 = nn.LSTM(64, 64, batch_first=True)
       self.dropout2 = nn.Dropout(0.2)
       self.fc = nn.Linear(64, 1)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x, _ = self.lstm1(x)
        x = self.dropout1(x)
        x, _ = self.lstm2(x)
        x = self.dropout2(x)
        last_output = x[:, -1, :]
        last_output = self.fc(last_output)

        return(last_output)

In [63]:
class MyDataset(Dataset):
    def __init__(self, data_x, data_y):
        self.data_x = torch.tensor(data_x, dtype=torch.float32)
        self.data_y = torch.tensor(data_y, dtype=torch.float32)

    def __len__(self):
        return len(self.data_x)

    def __getitem__(self, idx):
        return self.data_x[idx], self.data_y[idx]

In [ ]:
sensor_idd=[1,2,3,4,5,6,7]
for sensor in sensor_idd:
    print(sensor)
    sensor_id_A = "BID" + str(sensor)
    sensor_id=['BID1','BID2','BID3','BID4','BID5','BID6','BID7']
    sensor_id_src = [sensor_id_A]
    if sensor_id_A in sensor_id:
        sensor_id.remove(sensor_id_A)
    sensor_id_trg = sensor_id

    subject_gait_data_x_src = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_src = []      #存每個受測者的分數
    subject_gait_data_x_trg1 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg1 = []      #存每個受測者的分數
    subject_gait_data_x_trg2 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg2 = []      #存每個受測者的分數
    subject_gait_data_x_trg3 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg3 = []      #存每個受測者的分數
    subject_gait_data_x_trg4 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg4 = []      #存每個受測者的分數
    subject_gait_data_x_trg5 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg5 = []      #存每個受測者的分數
    subject_gait_data_x_trg6 = []      #存每個受測者以切完的步態資料
    subject_gait_data_y_trg6 = []      #存每個受測者的分數

    cols_to_drop_src = []
    cols_to_drop_trg1 = []
    cols_to_drop_trg2 = []
    cols_to_drop_trg3 = []
    cols_to_drop_trg4 = []
    cols_to_drop_trg5 = []
    cols_to_drop_trg6 = []

    for files in filename:
        gait_x_src = []
        gait_y_src = []
        gait_x_trg1 = []
        gait_y_trg1 = []
        gait_x_trg2 = []
        gait_y_trg2 = []
        gait_x_trg3 = []
        gait_y_trg3 = []
        gait_x_trg4 = []
        gait_y_trg4 = []
        gait_x_trg5 = []
        gait_y_trg5 = []
        gait_x_trg6 = []
        gait_y_trg6 = []
        data = pd.read_csv(DatasetPath + files)
        print(DatasetPath + files)


        "刪除夾角資料"
        data_col_name2 = data.columns
        for col_name2 in data_col_name2:
            if col_name2[0:5] == "Right" or col_name2[0:4] == "Left":
                data = data.drop(col_name2, axis = 1)

        "刪除其他sensor資料"
        data_col_name3 = data.columns
        for col_name3 in data_col_name3:
            if col_name3[0:3] == "BID" :
                if (col_name3[0:4] != sensor_id_A):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_src.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[0]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg1.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[1]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg2.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[2]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg3.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[3]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg4.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[4]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg5.append(col_name3)
                if (col_name3[0:4] != sensor_id_trg[5]):  #指定要留的sensor資料，其他刪除(單顆)
                    cols_to_drop_trg6.append(col_name3)

        data_src = data.drop(columns=cols_to_drop_src)
        data_trg1 = data.drop(columns=cols_to_drop_trg1)
        data_trg2 = data.drop(columns=cols_to_drop_trg2)
        data_trg3 = data.drop(columns=cols_to_drop_trg3)
        data_trg4 = data.drop(columns=cols_to_drop_trg4)
        data_trg5 = data.drop(columns=cols_to_drop_trg5)
        data_trg6 = data.drop(columns=cols_to_drop_trg6)

        row , colume = data.shape

        "取得量表分數"
        scale_score = y_data.loc[y_data['ID'] == int(files[0:3]) ,'score'].values
        print(scale_score)

        "找出每個資料的頭跟尾"
        task_ID = 'T'
        number = 1
        task_StartEnd = pd.DataFrame(columns=["task","start","end"])
        f=0
        i=0
        while i<row:
            if data.iloc[i,1] == task_ID and f == 0:
                f=1
                task_StartEnd.loc[number , "task"] = task_ID
                task_StartEnd.loc[number , "start"] = i
            elif f == 1 and data.iloc[i,1] !=task_ID:
                f=0
                task_StartEnd.loc[number , "end"] = i-1
                number += 1
                task_ID = chr(ord(task_ID) + 1)
                i-=1
            i+=1
        task_StartEnd.loc[number , "end"] = i-1

        "設定window size、前後筆資料重複率"
        window_size = 150       #sample rate = 50Hz，取一秒的資料
        Repeat_ratio = int(window_size * ((100-50)/100)) 
        "切割訓練資料"
        task_ID = 'T'
        for i in range(0,6):
            start = task_StartEnd.at[i+1 , 'start']
            end = task_StartEnd.at[i+1 , 'end'] + 1

            start = start + 50      #刪除開始後1秒的步態資料
            end = end - 50          #刪除結束前1秒的步態資料

            total = end - start
            training_set_src = data_src.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg1 = data_trg1.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg2 = data_trg2.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg3 = data_trg3.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg4 = data_trg4.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg5 = data_trg5.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            training_set_trg6 = data_trg6.iloc[start:end,5:colume].values #取7顆IMU的9軸資料，並且依照動作T、U、V、W、X、Y動作分段取
            "Z-score Standardization"
            training_set_src = stats.zscore(training_set_src)
            training_set_trg1 = stats.zscore(training_set_trg1)
            training_set_trg2 = stats.zscore(training_set_trg2)
            training_set_trg3 = stats.zscore(training_set_trg3)
            training_set_trg4 = stats.zscore(training_set_trg4)
            training_set_trg5 = stats.zscore(training_set_trg5)
            training_set_trg6 = stats.zscore(training_set_trg6)

            for j in range(0 , total , Repeat_ratio):
                if j + window_size <= total:
                    gait_x_src.append(training_set_src[j:j + window_size])
                    gait_y_src.append(scale_score)                   #切好的資料lable
                    gait_x_trg1.append(training_set_trg1[j:j + window_size])
                    gait_y_trg1.append(scale_score)                   #切好的資料lable
                    gait_x_trg2.append(training_set_trg2[j:j + window_size])
                    gait_y_trg2.append(scale_score)                   #切好的資料lable
                    gait_x_trg3.append(training_set_trg3[j:j + window_size])
                    gait_y_trg3.append(scale_score)                   #切好的資料lable
                    gait_x_trg4.append(training_set_trg4[j:j + window_size])
                    gait_y_trg4.append(scale_score)                   #切好的資料lable
                    gait_x_trg5.append(training_set_trg5[j:j + window_size])
                    gait_y_trg5.append(scale_score)                   #切好的資料lable
                    gait_x_trg6.append(training_set_trg6[j:j + window_size])
                    gait_y_trg6.append(scale_score)                   #切好的資料lable
        gait_x_src = np.array(gait_x_src)
        gait_y_src = np.array(gait_y_src)
        gait_x_trg1 = np.array(gait_x_trg1)
        gait_y_trg1 = np.array(gait_y_trg1)
        gait_x_trg2 = np.array(gait_x_trg2)
        gait_y_trg2 = np.array(gait_y_trg2)
        gait_x_trg3 = np.array(gait_x_trg3)
        gait_y_trg3 = np.array(gait_y_trg3)
        gait_x_trg4 = np.array(gait_x_trg4)
        gait_y_trg4 = np.array(gait_y_trg4)
        gait_x_trg5 = np.array(gait_x_trg5)
        gait_y_trg5 = np.array(gait_y_trg5)
        gait_x_trg6 = np.array(gait_x_trg6)
        gait_y_trg6 = np.array(gait_y_trg6)
        subject_gait_data_x_src.append(gait_x_src)
        subject_gait_data_y_src.append(scale_score)
        subject_gait_data_x_trg1.append(gait_x_trg1)
        subject_gait_data_x_trg2.append(gait_x_trg2)
        subject_gait_data_x_trg3.append(gait_x_trg3)
        subject_gait_data_x_trg4.append(gait_x_trg4)
        subject_gait_data_x_trg5.append(gait_x_trg5)
        subject_gait_data_x_trg6.append(gait_x_trg6)
        subject_gait_data_y_trg1.append(scale_score)
        subject_gait_data_y_trg2.append(scale_score)
        subject_gait_data_y_trg3.append(scale_score)
        subject_gait_data_y_trg4.append(scale_score)
        subject_gait_data_y_trg5.append(scale_score)
        subject_gait_data_y_trg6.append(scale_score)


    "打亂受測者的順序"
    r = list(zip(subject_gait_data_x_src,subject_gait_data_y_src,subject_gait_data_x_trg1,subject_gait_data_y_trg1,subject_gait_data_x_trg2,subject_gait_data_y_trg2,
                subject_gait_data_x_trg3,subject_gait_data_y_trg3,subject_gait_data_x_trg4,subject_gait_data_y_trg4,subject_gait_data_x_trg5,subject_gait_data_y_trg5,
                subject_gait_data_x_trg6,subject_gait_data_y_trg6))
    np.random.shuffle(r)
    (subject_gait_data_x_src,subject_gait_data_y_src,subject_gait_data_x_trg1,subject_gait_data_y_trg1,subject_gait_data_x_trg2,
    subject_gait_data_y_trg2,subject_gait_data_x_trg3,subject_gait_data_y_trg3,subject_gait_data_x_trg4,subject_gait_data_y_trg4,
    subject_gait_data_x_trg5,subject_gait_data_y_trg5,subject_gait_data_x_trg6,subject_gait_data_y_trg6) = zip(*r)

    "量表分數 normalization"
    subject_gait_data_y_src = np.array(subject_gait_data_y_src)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_src = scaler.transform(subject_gait_data_y_src)

    subject_gait_data_y_trg1 = np.array(subject_gait_data_y_trg1)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg1 = scaler.transform(subject_gait_data_y_trg1)

    subject_gait_data_y_trg2 = np.array(subject_gait_data_y_trg2)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg2 = scaler.transform(subject_gait_data_y_trg2)

    subject_gait_data_y_trg3 = np.array(subject_gait_data_y_trg3)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg3 = scaler.transform(subject_gait_data_y_trg3)

    subject_gait_data_y_trg4 = np.array(subject_gait_data_y_trg4)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg4 = scaler.transform(subject_gait_data_y_trg4)

    subject_gait_data_y_trg5 = np.array(subject_gait_data_y_trg5)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg5 = scaler.transform(subject_gait_data_y_trg5)

    subject_gait_data_y_trg6 = np.array(subject_gait_data_y_trg6)     #從別的資料型態轉成array
    BBS = [[0],[56]]                                        #BBS的分數範圍
    BBS_array = np.array(BBS)
    scaler = MinMaxScaler(feature_range=(0,1)).fit(BBS_array)
    subject_gait_data_y_trg6 = scaler.transform(subject_gait_data_y_trg6)

    savepath = 'save/'
    if not os.path.isdir(savepath):
        os.mkdir(savepath)
    savepath = savepath + 'S2T(1-1)/'
    if not os.path.isdir(savepath):
        os.mkdir(savepath)

    n=0
    epoch_range = [500]                             #設定epoch實驗參數
    batch_size_range = [64]



    for epochs in epoch_range:                    #實驗epoch
        for batch_size in batch_size_range:       #實驗batch size
            n=3
            localtime = time.localtime(time.time())
            savepath = 'save/S2T(1-1)/' +str(localtime[0])+str(localtime[1]).zfill(2)+str(localtime[2]).zfill(2)+'v'+str(n)+'_sensor'+str(sensor)+'/'
            if not os.path.isdir(savepath):
                os.mkdir(savepath)
            val_times = 5       # k fold cross validation
            MAE_average1 , MAE_average2 , MAE_average3 , MAE_average4 , MAE_average5 , MAE_average6 = 0,0,0,0,0,0
            times = 0
            subject_size = len(subject_gait_data_x_src)

            for val in range(val_times):
                times+=1

                x_train_src, y_train_src, x_test_src, y_test_src = split_and_shuffle(subject_gait_data_x_src, subject_gait_data_y_src,val_times,subject_size,val)

                x_train1, y_train1, x_test1, y_test1 = split_and_shuffle(subject_gait_data_x_trg1, subject_gait_data_y_trg1,val_times,subject_size,val)

                x_train2, y_train2, x_test2, y_test2 = split_and_shuffle(subject_gait_data_x_trg2, subject_gait_data_y_trg2,val_times,subject_size,val)
                
                x_train3, y_train3, x_test3, y_test3 = split_and_shuffle(subject_gait_data_x_trg3, subject_gait_data_y_trg3,val_times,subject_size,val)

                x_train4, y_train4, x_test4, y_test4 = split_and_shuffle(subject_gait_data_x_trg4, subject_gait_data_y_trg4,val_times,subject_size,val)

                x_train5, y_train5, x_test5, y_test5 = split_and_shuffle(subject_gait_data_x_trg5, subject_gait_data_y_trg5,val_times,subject_size,val)

                x_train6, y_train6, x_test6, y_test6 = split_and_shuffle(subject_gait_data_x_trg6, subject_gait_data_y_trg6,val_times,subject_size,val)



                src_train_dataset = MyDataset(x_train_src, y_train_src)
                src_test_dataset = MyDataset(x_test_src, y_test_src)
                trg_train_dataset1 = MyDataset(x_train1, y_train1)
                trg_test_dataset1 = MyDataset(x_test1, y_test1)
                trg_train_dataset2 = MyDataset(x_train2, y_train2)
                trg_test_dataset2 = MyDataset(x_test2, y_test2)
                trg_train_dataset3 = MyDataset(x_train3, y_train3)
                trg_test_dataset3 = MyDataset(x_test3, y_test3)
                trg_train_dataset4 = MyDataset(x_train4, y_train4)
                trg_test_dataset4 = MyDataset(x_test4, y_test4)
                trg_train_dataset5 = MyDataset(x_train5, y_train5)
                trg_test_dataset5 = MyDataset(x_test5, y_test5)
                trg_train_dataset6 = MyDataset(x_train6, y_train6)
                trg_test_dataset6 = MyDataset(x_test6, y_test6)

                src_train_loader = DataLoader(src_train_dataset, batch_size=64, shuffle=False)
                src_test_loader = DataLoader(src_test_dataset, batch_size=64, shuffle=False)
                trg_train_loader1 = DataLoader(trg_train_dataset1, batch_size=64, shuffle=False)
                trg_test_loader1 = DataLoader(trg_test_dataset1, batch_size=64, shuffle=False)
                trg_train_loader2 = DataLoader(trg_train_dataset2, batch_size=64, shuffle=False)
                trg_test_loader2 = DataLoader(trg_test_dataset2, batch_size=64, shuffle=False)
                trg_train_loader3 = DataLoader(trg_train_dataset3, batch_size=64, shuffle=False)
                trg_test_loader3 = DataLoader(trg_test_dataset3, batch_size=64, shuffle=False)
                trg_train_loader4 = DataLoader(trg_train_dataset4, batch_size=64, shuffle=False)
                trg_test_loader4 = DataLoader(trg_test_dataset4, batch_size=64, shuffle=False)
                trg_train_loader5 = DataLoader(trg_train_dataset5, batch_size=64, shuffle=False)
                trg_test_loader5 = DataLoader(trg_test_dataset5, batch_size=64, shuffle=False)
                trg_train_loader6 = DataLoader(trg_train_dataset6, batch_size=64, shuffle=False)
                trg_test_loader6 = DataLoader(trg_test_dataset6, batch_size=64, shuffle=False)


                # 訓練部分
                F = FeatureExtractor().to(DEVICE)
                C = Classifier().to(DEVICE)
                optimizer = optim.Adam(list(F.parameters()) + list(C.parameters()),lr=0.0005)
                criterion = nn.MSELoss()
                criterion_mae = nn.L1Loss()

                for epoch in range(epochs):
                    F.train()
                    C.train()
                    for i,data in enumerate(src_train_loader):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        label=y_batch.to(DEVICE)
                        optimizer.zero_grad()
                        feature = F(input)
                        outputs = C(feature)
                        loss = criterion(outputs, label)
                        loss.backward()
                        optimizer.step()

                # 保存模型
                feature_extractor_filename = f'FeatureExtractor_fold{val}.pth'
                classifier_filename = f'Classifier_fold{val}.pth'
                torch.save(F.state_dict(), os.path.join(savepath, feature_extractor_filename))
                torch.save(C.state_dict(), os.path.join(savepath, classifier_filename))

                
                with torch.no_grad():
                    F.eval()
                    C.eval()
                    test_loss1 ,test_loss2 ,test_loss3 ,test_loss4 ,test_loss5 ,test_loss6 = 0.0 ,0.0 ,0.0 ,0.0 ,0.0 ,0.0 
                    for i,data in enumerate(trg_test_loader1):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss1 += loss.item()
                    test_loss1 /= len(trg_test_loader1)
                    print(f'Test Loss1: {test_loss1:.4f}')
                    MAE_average1 = MAE_average1 + test_loss1

                    for i,data in enumerate(trg_test_loader2):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss2 += loss.item()
                    test_loss2 /= len(trg_test_loader2)
                    print(f'Test Loss2: {test_loss2:.4f}')
                    MAE_average2 = MAE_average2 + test_loss2

                    for i,data in enumerate(trg_test_loader3):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss3 += loss.item()
                    test_loss3 /= len(trg_test_loader3)
                    print(f'Test Loss3: {test_loss3:.4f}')
                    MAE_average3 = MAE_average3 + test_loss3

                    for i,data in enumerate(trg_test_loader4):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss4 += loss.item()
                    test_loss4 /= len(trg_test_loader4)
                    print(f'Test Loss4: {test_loss4:.4f}')
                    MAE_average4 = MAE_average4 + test_loss4

                    for i,data in enumerate(trg_test_loader5):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss5 += loss.item()
                    test_loss5 /= len(trg_test_loader5)
                    print(f'Test Loss5: {test_loss5:.4f}')
                    MAE_average5 = MAE_average5 + test_loss5

                    for i,data in enumerate(trg_test_loader6):
                        x_batch, y_batch = data
                        input=x_batch.to(DEVICE)
                        label=y_batch.to(DEVICE)
                        input = input.permute(0,2,1)
                        feature = F(input)
                        outputs = C(feature)
                        "將量表分數轉換成原來分數"
                        y_test_tran = label.cpu().detach().numpy()
                        y_pred_tran = outputs.cpu().detach().numpy()

                        y_test_tran = scaler.inverse_transform(y_test_tran)
                        y_pred_tran = scaler.inverse_transform(y_pred_tran)
                        y_pred_tran =np.around(y_pred_tran)

                        y_test_tran_tensor = torch.tensor(y_test_tran, dtype=torch.float32).to(DEVICE)
                        y_pred_tran_tensor = torch.tensor(y_pred_tran, dtype=torch.float32).to(DEVICE)
                        loss = criterion_mae(y_pred_tran_tensor, y_test_tran_tensor)
                        test_loss6 += loss.item()
                    test_loss6 /= len(trg_test_loader6)
                    print(f'Test Loss6: {test_loss6:.4f}')
                    MAE_average6 = MAE_average6 + test_loss6

            print("BBS MAE:")
            print(MAE_average1/val_times)
            print(MAE_average2/val_times)
            print(MAE_average3/val_times)
            print(MAE_average4/val_times)
            print(MAE_average5/val_times)
            print(MAE_average6/val_times)   
            f = open(savepath + '實驗結果.txt','a')
            f.write('window size = '+ str(window_size)+'\n')
            f.write('epoch = ' + str(epochs) + '\n')
            f.write('batch size = ' + str(batch_size) + '\n')
            f.write('MAE1 = ' + str(MAE_average1/val_times) + '\n')
            f.write('MAE2 = ' + str(MAE_average2/val_times) + '\n')
            f.write('MAE3 = ' + str(MAE_average3/val_times) + '\n')
            f.write('MAE4 = ' + str(MAE_average4/val_times) + '\n')
            f.write('MAE5 = ' + str(MAE_average5/val_times) + '\n')
            f.write('MAE6 = ' + str(MAE_average6/val_times) + '\n')
            f.close()


